# Tech Challenge Fase 3 - Fine-Tuning Foundation Model

## Geração de Descrições de Produtos com Phi-4 e AmazonTitles-1.3MM

Este notebook implementa o fine-tuning do modelo **Phi-4-mini-instruct** (3.8B parâmetros) utilizando o dataset **AmazonTitles-1.3MM** para geração automática de descrições de produtos a partir de títulos.

**Objetivo**: Criar um modelo capaz de receber um título de produto e gerar uma descrição detalhada, coerente e relevante no estilo Amazon.

**Técnica**: LoRA (Low-Rank Adaptation) - Parameter-Efficient Fine-Tuning

**Hardware**: NVIDIA GPU com 16GB+ VRAM (testado em RTX 2000 Ada)

## Instalação de Dependências

Instala todas as bibliotecas necessárias a partir do arquivo `requirements.txt`.

In [1]:
!pip install -q -r requirements.txt

## Detecção de Ambiente e Configuração de Diretórios

Identifica se está executando no Google Colab ou ambiente local e configura os caminhos dinamicamente.

## Configuração Inicial

Este notebook pode ser executado tanto **localmente** quanto no **Google Colab**.

### Execução Local

1. **Instale as dependências**: `pip install -r requirements.txt`
2. **Baixe o dataset**: [AmazonTitles-1.3MM](https://drive.google.com/file/d/12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK/view)
3. **Coloque o arquivo** `trn.json` na pasta `dataset/`
4. **Execute as células sequencialmente**

### Execução no Google Colab

1. **Faça upload do arquivo** `trn.json` para o Google Drive
2. **Ajuste o caminho** `BASE_DIR` na célula de configuração (próxima célula) para corresponder ao seu Drive
3. **Ative GPU**: Runtime > Change runtime type > GPU (T4 ou superior)
4. **Execute as células sequencialmente**

**Tempo estimado de execução**: ~40-60 minutos (com GPU NVIDIA)

In [2]:
import sys
import os

# Verifica se está no Google Colab
IS_COLAB = 'google.colab' in sys.modules

if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    # Define o diretório base no Google Drive
    # ATENÇÃO: Ajuste este caminho para o local onde o seu projeto está no Google Drive
    BASE_DIR = '/content/drive/MyDrive/POS/TC-3/'
    print(f"✓ Executando no Google Colab")
    print(f"✓ Diretório base: {BASE_DIR}")
else:
    # Usa o diretório local se não estiver no Colab
    BASE_DIR = './'
    print("✓ Executando em ambiente local")

# Cria os diretórios se não existirem, prefixados com o BASE_DIR
os.makedirs(os.path.join(BASE_DIR, 'dataset'), exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, 'outputs'), exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, 'modelo_finetuned'), exist_ok=True)
os.makedirs(os.path.join(BASE_DIR, 'checkpoints'), exist_ok=True)

# Define os caminhos dinamicamente
DATASET_PATH = os.path.join(BASE_DIR, 'dataset/trn.json')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')
CHECKPOINT_DIR = os.path.join(BASE_DIR, 'checkpoints')
FINAL_MODEL_DIR = os.path.join(BASE_DIR, 'modelo_finetuned')

print(f"\n✓ Diretórios configurados:")
print(f"  - Dataset: {DATASET_PATH}")
print(f"  - Outputs: {OUTPUT_DIR}")
print(f"  - Checkpoints: {CHECKPOINT_DIR}")
print(f"  - Modelo Final: {FINAL_MODEL_DIR}")

✓ Executando em ambiente local

✓ Diretórios configurados:
  - Dataset: ./dataset/trn.json
  - Outputs: ./outputs
  - Checkpoints: ./checkpoints
  - Modelo Final: ./modelo_finetuned


## 1. Configuração Inicial

Instalação de dependências e configuração do ambiente.

**Objetivo**: Preparar o ambiente Python com todas as bibliotecas necessárias para o fine-tuning.

**Bibliotecas principais**:
- `torch`: Framework de deep learning
- `transformers`: Biblioteca Hugging Face para modelos de linguagem
- `datasets`: Manipulação eficiente de datasets
- `peft`: Parameter-Efficient Fine-Tuning (LoRA)
- `accelerate`: Otimização de treinamento distribuído
- `pandas`, `numpy`: Manipulação de dados
- `scikit-learn`: Split de dados e métricas

### Verificação da Versão do Python

Exibe informações sobre o interpretador Python em uso no kernel do Jupyter.

In [3]:
# Verificar versão do Python do kernel
import sys

print("\n" + "="*80)
print("INFORMAÇÕES DO SISTEMA")
print("="*80)
print(f"Python: {sys.version.split()[0]}")
print(f"Executável: {sys.executable}")
print("="*80 + "\n")


INFORMAÇÕES DO SISTEMA
Python: 3.12.9
Executável: /bin/python3.12



### Importação de Bibliotecas e Verificação de GPU

Importa todas as bibliotecas necessárias e verifica disponibilidade de GPU CUDA.

In [4]:
# Imports
import json
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ML/DL
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Utils
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

print("\n" + "="*80)
print("AMBIENTE PYTORCH")
print("="*80)
print(f"PyTorch: {torch.__version__}")
print(f"CUDA disponível: {'Sim' if torch.cuda.is_available() else 'Não'}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
print("="*80 + "\n")


AMBIENTE PYTORCH
PyTorch: 2.8.0+cu128
CUDA disponível: Sim
GPU: NVIDIA RTX 2000 Ada Generation
VRAM: 16.0 GB



### Definição de Hiperparâmetros e Configurações

Configura todas as constantes e hiperparâmetros do projeto: modelo, caminhos, épocas, batch size, LoRA, etc.

In [5]:
# --- Configurações Gerais ---
MODEL_NAME = "microsoft/Phi-4-mini-instruct"
SAMPLE_SIZE = 20000     
SEED = 42

# --- Configuração de Precisão ---
TORCH_DTYPE = torch.float16 

# --- Hiperparâmetros (otimizado para RTX 2000 Ada) ---
EPOCHS = 2 
BATCH_SIZE = 2
LEARNING_RATE = 5e-5
MAX_LENGTH = 128

# --- LoRA ---
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.1

# Exibir configurações
print("\n" + "="*80)
print("CONFIGURAÇÕES DO PROJETO")
print("="*80)

print("\n--- Modelo e Caminhos ---")
print(f"Modelo: {MODEL_NAME}")
print(f"Dataset: {DATASET_PATH}")
print(f"Output: {OUTPUT_DIR}")
print(f"Modelo Final: {FINAL_MODEL_DIR}")

print("\n--- Hiperparâmetros de Treinamento ---")
print(f"Épocas: {EPOCHS}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Batch Size Efetivo: {BATCH_SIZE * 4} (com gradient accumulation)")
print(f"Learning Rate: {LEARNING_RATE:.0e}")
print(f"Max Length: {MAX_LENGTH} tokens")
print(f"Precisão: {str(TORCH_DTYPE).split('.')[-1]}")

print("\n--- Configurações LoRA ---")
print(f"Rank (r): {LORA_R}")
print(f"Alpha: {LORA_ALPHA}")
print(f"Dropout: {LORA_DROPOUT}")

print("\n--- Dataset ---")
print(f"Tamanho da Amostra: {SAMPLE_SIZE:,}")
print(f"Seed: {SEED}")
print(f"Tempo Estimado: 40-60 minutos (2 épocas)")

print("="*80 + "\n")


CONFIGURAÇÕES DO PROJETO

--- Modelo e Caminhos ---
Modelo: microsoft/Phi-4-mini-instruct
Dataset: ./dataset/trn.json
Output: ./outputs
Modelo Final: ./modelo_finetuned

--- Hiperparâmetros de Treinamento ---
Épocas: 2
Batch Size: 2
Batch Size Efetivo: 8 (com gradient accumulation)
Learning Rate: 5e-05
Max Length: 128 tokens
Precisão: float16

--- Configurações LoRA ---
Rank (r): 16
Alpha: 32
Dropout: 0.1

--- Dataset ---
Tamanho da Amostra: 20,000
Seed: 42
Tempo Estimado: 40-60 minutos (2 épocas)



### Hiperparâmetros Otimizados

Os parâmetros foram ajustados especificamente para a GPU NVIDIA RTX 2000 Ada (16GB):

**Configurações de Dataset**:
- `SAMPLE_SIZE = 20000`: Amostra reduzida para tempo de treinamento viável (~40-60min)
- `MAX_LENGTH = 128`: Tokens por sequência (balance entre contexto e performance)

**Configurações de Treinamento**:
- `EPOCHS = 2`: Duas passagens completas pelo dataset
- `BATCH_SIZE = 2`: Amostras processadas simultaneamente por dispositivo
- `LEARNING_RATE = 5e-5`: Taxa de aprendizado recomendada para Phi-4
- `TORCH_DTYPE = torch.float16`: FP16 para treinamento rápido e eficiente

**Configurações LoRA**:
- `LORA_R = 16`: Rank das matrizes de adaptação (maior = mais expressivo)
- `LORA_ALPHA = 32`: Fator de escala (geralmente 2× do rank)
- `LORA_DROPOUT = 0.1`: Regularização para evitar overfitting

## 2. Preparação do Dataset

Carregamento, exploração e processamento do dataset AmazonTitles-1.3MM.

**Dataset**: AmazonTitles-1.3MM contém 2.2M+ registros de produtos Amazon com títulos e descrições.

**Pipeline de preparação**:
1. **Carregamento**: Leitura do arquivo JSON linha por linha
2. **Análise exploratória**: Estatísticas descritivas e qualidade dos dados
3. **Limpeza**: Remoção de nulos, duplicatas e registros com conteúdo muito curto (750K registros com content vazio removidos)
4. **Amostragem**: Seleção aleatória de 20.000 registros (1.52% do dataset limpo) para viabilizar treinamento
5. **Formatação**: Criação de prompts estruturados para o modelo
6. **Split**: Divisão em 90% treino (18.000) e 10% validação (2.000)

### Carregamento e Validação do Dataset

Verifica se o arquivo `trn.json` existe e carrega os dados em um DataFrame pandas.

In [6]:
# Validar existência do dataset
from pathlib import Path

print("\n" + "="*80)
print("CARREGAMENTO DO DATASET")
print("="*80)

dataset_file = Path(DATASET_PATH)
if not dataset_file.exists():
    print(f"❌ ERRO: Dataset não encontrado em: {DATASET_PATH}")
    raise FileNotFoundError(
        f"\nSe estiver no Google Colab, certifique-se de:\n"
        f"1. Fazer upload do arquivo 'trn.json' para o Google Drive\n"
        f"2. Ajustar o caminho BASE_DIR na célula 2\n"
        f"3. Garantir que o arquivo está em: {DATASET_PATH}"
    )

print(f"✓ Dataset encontrado: {DATASET_PATH}")

# Carregar dataset (apenas colunas necessárias)
print("⏳ Carregando dataset...")

data = []
with open(DATASET_PATH, 'r', encoding='utf-8') as f:
    for line in f:
        record = json.loads(line)
        # Manter apenas colunas úteis: title e content
        data.append({
            'title': record.get('title', ''),
            'content': record.get('content', '')
        })

df = pd.DataFrame(data)

print(f"✓ Dataset carregado: {len(df):,} registros")
print(f"ℹ Colunas disponíveis: {', '.join(df.columns.tolist())}")

print("\nPrimeiras 5 linhas:")
print(df.head())
print("="*80 + "\n")


CARREGAMENTO DO DATASET
✓ Dataset encontrado: ./dataset/trn.json
⏳ Carregando dataset...
✓ Dataset carregado: 2,248,619 registros
ℹ Colunas disponíveis: title, content

Primeiras 5 linhas:
                                               title  \
0                        Girls Ballet Tutu Neon Pink   
1                           Adult Ballet Tutu Yellow   
2  The Way Things Work: An Illustrated Encycloped...   
3                                      Mog's Kittens   
4                              Misty of Chincoteague   

                                             content  
0  High quality 3 layer ballet tutu. 12 inches in...  
1                                                     
2                                                     
3  Judith Kerr&#8217;s best&#8211;selling adventu...  
4                                                     



**Resultado Esperado:**
- Dataset carregado com sucesso: 2.248.619 registros (2.2M+)
- Primeiras 5 linhas exibidas com colunas `title` e `content`
- Validação de integridade: arquivo JSON válido linha por linha
- 749.901 registros identificados com content vazio (serão removidos na limpeza)

### Análise Exploratória dos Dados

Exibe estatísticas descritivas do dataset: shape, valores nulos, comprimento de títulos e descrições.

In [7]:
# Análise exploratória simplificada
print("\n--- Análise Exploratória ---")

print(f"Shape: {df.shape[0]:,} linhas × {df.shape[1]} colunas")
print(f"Valores nulos (title): {df['title'].isnull().sum()}")
print(f"Valores nulos (content): {df['content'].isnull().sum()}")

print("\nEstatísticas de comprimento:")
title_stats = df['title'].str.len()
content_stats = df['content'].str.len()

print(f"{'Campo':<10} {'Mínimo':<10} {'Máximo':<10} {'Média':<10}")
print("-" * 45)
print(f"{'Title':<10} {title_stats.min():<10} {title_stats.max():<10} {title_stats.mean():<10.1f}")
print(f"{'Content':<10} {content_stats.min():<10} {content_stats.max():<10} {content_stats.mean():<10.1f}")

empty_content = (df['content'].str.len() == 0).sum()
if empty_content > 0:
    print(f"\n⚠ Registros com content vazio: {empty_content:,}")


--- Análise Exploratória ---
Shape: 2,248,619 linhas × 2 colunas
Valores nulos (title): 0
Valores nulos (content): 0

Estatísticas de comprimento:
Campo      Mínimo     Máximo     Média     
---------------------------------------------
Title      0          1999       55.3      
Content    0          197622     415.0     

⚠ Registros com content vazio: 749,901


### Limpeza e Filtragem de Dados

Remove registros com valores nulos, duplicatas e textos muito curtos para garantir qualidade dos dados de treinamento.

In [8]:
# Limpeza de dados
print("\n--- Limpeza de Dados ---")
print("⏳ Aplicando filtros de qualidade...")

original_count = len(df)

# Remover nulos nas colunas principais
df_clean = df.dropna(subset=['title', 'content']).copy()

# Remover duplicatas
df_clean = df_clean.drop_duplicates(subset=['title'])

# Remover registros com conteúdo muito curto
df_clean = df_clean[
    (df_clean['title'].str.len() > 10) & 
    (df_clean['content'].str.len() > 20)
]

clean_count = len(df_clean)
retention_rate = (clean_count / original_count) * 100

print(f"✓ Limpeza concluída: {clean_count:,} registros ({retention_rate:.1f}% retidos)")
print(f"ℹ Registros removidos: {original_count - clean_count:,}")


--- Limpeza de Dados ---
⏳ Aplicando filtros de qualidade...
✓ Limpeza concluída: 1,315,931 registros (58.5% retidos)
ℹ Registros removidos: 932,688


### Amostragem Aleatória do Dataset

Seleciona uma amostra aleatória de 3000 registros para viabilizar o treinamento em tempo razoável.

In [9]:
# Amostragem (para viabilizar treinamento)
print("\n--- Amostragem do Dataset ---")

if len(df_clean) > SAMPLE_SIZE:
    df_sample = df_clean.sample(n=SAMPLE_SIZE, random_state=SEED)
    print(f"ℹ Amostra aleatória selecionada: {SAMPLE_SIZE:,} registros")
    print(f"Taxa de amostragem: {(SAMPLE_SIZE/len(df_clean)*100):.2f}%")
else:
    df_sample = df_clean
    print(f"ℹ Dataset completo utilizado: {len(df_sample):,} registros")


--- Amostragem do Dataset ---
ℹ Amostra aleatória selecionada: 20,000 registros
Taxa de amostragem: 1.52%


### Formatação de Prompts para Fine-Tuning

Cria prompts estruturados no formato esperado pelo modelo, combinando título e descrição.

In [10]:
# Formatação para fine-tuning (prompt engineering)
print("\n--- Formatação de Prompts ---")

def create_prompt(title, content):
    """Cria prompt no formato esperado pelo modelo"""
    return f"""Descreva o produto com o seguinte título: {title}

Descrição: {content}"""

df_sample['prompt'] = df_sample.apply(
    lambda row: create_prompt(row['title'], row['content']), 
    axis=1
)

print(f"✓ Prompts formatados: {len(df_sample):,} registros")

print("\nExemplo de prompt formatado:")
print("-" * 80)
print(df_sample['prompt'].iloc[0][:300] + "...")
print("-" * 80)


--- Formatação de Prompts ---
✓ Prompts formatados: 20,000 registros

Exemplo de prompt formatado:
--------------------------------------------------------------------------------
Descreva o produto com o seguinte título: Merrick Before Grain #2 Salmon Dry Cat Food, 1-Pound Bag

Descrição: Before Grain Salmon #2 - 3.3-Pound bag: Grain-free formula made with Salmon. Meat is #1 Ingredient. Made in the USA. Grain Free....
--------------------------------------------------------------------------------


### Divisão em Conjuntos de Treino e Validação

Divide o dataset em 90% para treinamento e 10% para validação, convertendo para formato Hugging Face Dataset.

In [11]:
# Split treino/validação
print("\n--- Split Treino/Validação ---")

train_df, val_df = train_test_split(
    df_sample, 
    test_size=0.1, 
    random_state=SEED
)

print(f"Treino: {len(train_df):,} registros (90%)")
print(f"Validação: {len(val_df):,} registros (10%)")

# Converter para Dataset do Hugging Face
train_dataset = Dataset.from_pandas(train_df[['prompt']])
val_dataset = Dataset.from_pandas(val_df[['prompt']])

print("✓ Datasets preparados e prontos para tokenização!")


--- Split Treino/Validação ---
Treino: 18,000 registros (90%)
Validação: 2,000 registros (10%)
✓ Datasets preparados e prontos para tokenização!


## 3. Teste do Modelo Base (Baseline)

Carregamento e teste do modelo foundation antes do treinamento para estabelecer baseline.

**Modelo**: microsoft/Phi-4-mini-instruct
- **Parâmetros**: 3.8B
- **Contexto**: 128K tokens
- **Arquitetura**: Transformer decoder-only otimizado

**Objetivo do baseline**:
- Avaliar capacidade inicial do modelo sem fine-tuning
- Estabelecer métricas de comparação pré/pós treinamento
- Verificar qualidade das gerações antes da especialização

**Metodologia**:
- Seleção aleatória de 3 amostras do conjunto de validação
- Geração de descrições com temperatura 0.7
- Análise qualitativa das respostas

### Carregamento do Modelo Foundation e Tokenizer

Baixa e carrega o modelo Phi-4-mini-instruct e seu tokenizer da Hugging Face.

In [12]:
# Carregar modelo e tokenizer base
print("\n" + "="*80)
print("CARREGAMENTO DO MODELO BASE")
print("="*80)
print(f"⏳ Baixando/carregando {MODEL_NAME}...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=TORCH_DTYPE if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

print("✓ Modelo base carregado com sucesso!")
print(f"Parâmetros totais: {base_model.num_parameters():,}")
print(f"Dtype: {str(TORCH_DTYPE if torch.cuda.is_available() else torch.float32).split('.')[-1]}")
print(f"Device: {'GPU (auto)' if torch.cuda.is_available() else 'CPU'}")
print("="*80 + "\n")


CARREGAMENTO DO MODELO BASE
⏳ Baixando/carregando microsoft/Phi-4-mini-instruct...


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.03s/it]


✓ Modelo base carregado com sucesso!
Parâmetros totais: 3,836,021,760
Dtype: float16
Device: GPU (auto)



### Função de Geração de Texto

Define função auxiliar para gerar respostas do modelo dado um prompt de entrada.

In [13]:
# Função para gerar resposta
def generate_response(model, prompt, max_new_tokens=100):
    """Gera resposta do modelo dado um prompt"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.3,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id
        )
    
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

### Testes com Modelo Base (Baseline)

Testa o modelo original com 3 amostras aleatórias para estabelecer baseline de comparação.

In [14]:
# Testes com modelo base
print("\n--- Testes com Modelo Base (Baseline) ---")
print("ℹ Selecionando 3 amostras aleatórias para teste")

test_samples = val_df.sample(n=3, random_state=SEED)

print("\n" + "="*80)
print("TESTES COM MODELO BASE (PRÉ FINE-TUNING)")
print("="*80)

for i, (idx, row) in enumerate(test_samples.iterrows(), 1):
    test_prompt = f"Descreva o produto com o seguinte título: {row['title']}\n\nDescrição:"
    
    print("\n" + "="*80)
    print(f"Teste {i}/3")
    print("="*80)
    print(f"Título: {row['title']}")
    
    print(f"\nDescrição Real:")
    print(f"{row['content'][:200]}...\n")
    
    print("⏳ Gerando resposta do modelo base...")
    response = generate_response(base_model, test_prompt)
    print(f"\nResposta do Modelo Base:")
    print(response)
    print()

# Salvar resultados para comparação posterior
base_results = test_samples.copy()
print(f"✓ Baseline de {len(base_results)} amostras salvo para comparação posterior")


--- Testes com Modelo Base (Baseline) ---
ℹ Selecionando 3 amostras aleatórias para teste

TESTES COM MODELO BASE (PRÉ FINE-TUNING)

Teste 1/3
Título: The Divine Liturgy: A Commentary in the Light of the Fathers

Descrição Real:
Hieromonk Gregorios(in the world Panagiotis Hatziemmanuel) was born in Mytilene in 1936. He studied theology at the University of Athens, and then pursued graduate studies in area of Patristic Theolog...

⏳ Gerando resposta do modelo base...

Resposta do Modelo Base:
Descreva o produto com o seguinte título: The Divine Liturgy: A Commentary in the Light of the Fathers

Descrição: "The Divine Liturgy: A Commentary in the Light of the Fathers" é um estudo abrangente que oferece uma exploração detalhada da Liturgia Divina, o principal serviço de culto na Igreja Ortodoxa. O livro se baseia nos ensinamentos e interpretações dos Padres da Igreja, proporcionando uma compreensão profunda dos rituais, textos e significados teológicos envolvidos na Liturquia. Ele aborda

## 4. Fine-Tuning com LoRA/PEFT

Configuração e execução do fine-tuning utilizando LoRA para eficiência computacional.

**LoRA (Low-Rank Adaptation)**:
- Técnica de Parameter-Efficient Fine-Tuning (PEFT)
- Treina apenas 0.0819% dos parâmetros (3.145.728 de 3.839.167.488)
- Reduz drasticamente memória GPU e tempo de treinamento
- Mantém qualidade comparável ao fine-tuning completo

**Configuração**:
- **Rank (r)**: 16 - Dimensionalidade das matrizes LoRA
- **Alpha**: 32 - Fator de escala
- **Dropout**: 0.1 - Regularização
- **Target modules**: q_proj, k_proj, v_proj, o_proj (camadas de atenção)

**Hiperparâmetros de treinamento**:
- **Épocas**: 2
- **Batch size efetivo**: 8 (2 per device × 4 gradient accumulation)
- **Learning rate**: 5e-5
- **Precision**: FP16 para otimização de memória
- **Max length**: 128 tokens
- **Estratégia**: Salvamento e avaliação a cada 100 steps

**Resultados do treinamento**:
- **Duração**: ~40-60 minutos
- **Total Steps**: 4.500 (2.250 steps por época)
- **Épocas**: 2.0 completas
- **Loss final**: 2.7119
- **Checkpoints**: 45 salvos automaticamente (a cada 100 steps)
- **Melhor modelo**: Selecionado automaticamente por menor validation loss

### Configuração LoRA

Define os parâmetros LoRA (rank, alpha, dropout) e módulos alvo para fine-tuning eficiente.

In [15]:
# Configuração LoRA para Phi-4
print("\n--- Configuração LoRA ---")

lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Phi-4 attention modules
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

print("✓ Configuração LoRA criada com sucesso!")
print(f"Rank (r): {LORA_R}")
print(f"Alpha: {LORA_ALPHA}")
print(f"Dropout: {LORA_DROPOUT}")
print(f"Target Modules: q_proj, k_proj, v_proj, o_proj")


--- Configuração LoRA ---
✓ Configuração LoRA criada com sucesso!
Rank (r): 16
Alpha: 32
Dropout: 0.1
Target Modules: q_proj, k_proj, v_proj, o_proj


### Preparação do Modelo com LoRA

Recarrega o modelo base e aplica os adaptadores LoRA para treinamento eficiente de parâmetros.

In [16]:
# Preparar modelo para fine-tuning (recarregar para evitar conflitos)
print("\n--- Preparação do Modelo com LoRA ---")
print("⏳ Carregando modelo base novamente...")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=TORCH_DTYPE,
    trust_remote_code=True
)

# Mover modelo para GPU se disponível
if torch.cuda.is_available():
    model = model.cuda()
    print("ℹ Modelo movido para GPU")

# Habilitar gradientes
model.enable_input_require_grads()

# Aplicar LoRA
model = get_peft_model(model, lora_config)

print("✓ Modelo preparado para fine-tuning com LoRA!")
print(f"Dtype: {str(TORCH_DTYPE).split('.')[-1]}")

# Exibir estatísticas de parâmetros treináveis
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
trainable_percent = 100 * trainable_params / all_params

print(f"Parâmetros treináveis: {trainable_params:,}")
print(f"Total de parâmetros: {all_params:,}")
print(f"Percentual treinável: {trainable_percent:.4f}%")


--- Preparação do Modelo com LoRA ---
⏳ Carregando modelo base novamente...


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.01it/s]


ℹ Modelo movido para GPU
✓ Modelo preparado para fine-tuning com LoRA!
Dtype: float16
Parâmetros treináveis: 3,145,728
Total de parâmetros: 3,839,167,488
Percentual treinável: 0.0819%


### Tokenização dos Datasets

Converte os textos em tokens numéricos que o modelo pode processar, aplicando truncamento e padding.

In [17]:
# Tokenizar datasets
print("\n--- Tokenização dos Datasets ---")
print("⏳ Tokenizando amostras de treino e validação...")

def tokenize_function(examples):
    return tokenizer(
        examples["prompt"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length"
    )

tokenized_train = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_val = val_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=val_dataset.column_names
)

print("✓ Tokenização concluída!")
print(f"Treino tokenizado: {len(tokenized_train):,} amostras")
print(f"Validação tokenizada: {len(tokenized_val):,} amostras")


--- Tokenização dos Datasets ---
⏳ Tokenizando amostras de treino e validação...


Map: 100%|██████████| 2000/2000 [00:00<00:00, 16024.97 examples/s]

✓ Tokenização concluída!
Treino tokenizado: 18,000 amostras
Validação tokenizada: 2,000 amostras


### Configuração dos Argumentos de Treinamento

Define todos os hiperparâmetros do treinamento: épocas, batch size, learning rate, estratégias de salvamento, etc.

In [18]:
print("\n--- Configuração dos Argumentos de Treinamento ---")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=4,
    learning_rate=LEARNING_RATE,
    logging_steps=10,
    save_strategy="steps",
    save_steps=100,
    evaluation_strategy="steps",
    eval_steps=100,
    fp16=True,
    remove_unused_columns=False,
    report_to=[],
    dataloader_num_workers=0,
    load_best_model_at_end=True,
    metric_for_best_model="loss"
)

print("✓ Argumentos de treinamento configurados!")
print(f"Épocas: {EPOCHS}")
print(f"Batch Size por Device: {BATCH_SIZE}")
print(f"Gradient Accumulation Steps: 4")
print(f"Batch Size Efetivo: {BATCH_SIZE * 4}")
print(f"Learning Rate: {LEARNING_RATE:.0e}")
print(f"Logging Interval: 10 steps")
print(f"Save Interval: 100 steps")
print(f"Eval Interval: 100 steps")
print(f"Mixed Precision: FP16")
print(f"Dataloader Workers: 0")


--- Configuração dos Argumentos de Treinamento ---
✓ Argumentos de treinamento configurados!
Épocas: 2
Batch Size por Device: 2
Gradient Accumulation Steps: 4
Batch Size Efetivo: 8
Learning Rate: 5e-05
Logging Interval: 10 steps
Save Interval: 100 steps
Eval Interval: 100 steps
Mixed Precision: FP16
Dataloader Workers: 0


### Inicialização do Trainer

Cria o objeto Trainer do Hugging Face com modelo, datasets, argumentos e data collator configurados.

In [19]:
# Data collator
print("\n--- Configuração do Trainer ---")

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
)

print("✓ Trainer configurado e pronto para iniciar o treinamento!")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



--- Configuração do Trainer ---
✓ Trainer configurado e pronto para iniciar o treinamento!


### Execução do Fine-Tuning

Inicia o processo de treinamento do modelo. Esta etapa leva aproximadamente 40-60 minutos.

In [20]:
# Executar fine-tuning
print("\n" + "="*80)
print("INICIANDO FINE-TUNING")
print("="*80)
print("ℹ Este processo levará aproximadamente 40-60 minutos")
print("ℹ O progresso será exibido abaixo")

trainer.train()

print("\n" + "="*80)
print("FINE-TUNING CONCLUÍDO!")
print("="*80)
print("✓ Treinamento finalizado com sucesso")
print("ℹ Verificando métricas finais...")

# Exibir loss final se disponível
if trainer.state.log_history:
    final_metrics = trainer.state.log_history[-1]
    if 'loss' in final_metrics:
        print(f"Loss final: {final_metrics['loss']:.4f}")
print("="*80 + "\n")


INICIANDO FINE-TUNING
ℹ Este processo levará aproximadamente 40-60 minutos
ℹ O progresso será exibido abaixo


Step,Training Loss,Validation Loss
100,2.850200,2.835318
200,2.847500,2.803639
300,2.777600,2.788291
400,2.771700,2.782315
500,2.822500,2.772506
600,2.674000,2.768760
700,2.781400,2.765096
800,2.676900,2.761694
900,2.704700,2.758704
1000,2.700800,2.756218



FINE-TUNING CONCLUÍDO!
✓ Treinamento finalizado com sucesso
ℹ Verificando métricas finais...



### Salvamento do Modelo Fine-Tunado

Salva os adaptadores LoRA treinados e o tokenizer no diretório especificado.

In [21]:
# Salvar modelo fine-tunado
print("\n--- Salvando Modelo Fine-Tunado ---")
print(f"⏳ Salvando em {FINAL_MODEL_DIR}...")

model.save_pretrained(FINAL_MODEL_DIR)
tokenizer.save_pretrained(FINAL_MODEL_DIR)

print(f"✓ Modelo salvo com sucesso!")
print(f"ℹ Localização: {FINAL_MODEL_DIR}")
print(f"ℹ Arquivos salvos: adapter_config.json, adapter_model.safetensors, tokenizer")


--- Salvando Modelo Fine-Tunado ---
⏳ Salvando em ./modelo_finetuned...
✓ Modelo salvo com sucesso!
ℹ Localização: ./modelo_finetuned
ℹ Arquivos salvos: adapter_config.json, adapter_model.safetensors, tokenizer


## 5. Avaliação e Demonstração

Geração de respostas com o modelo fine-tunado.

**Objetivo**: Demonstrar a capacidade do modelo treinado de gerar descrições de produtos a partir de títulos.

**Metodologia**:
- Uso das mesmas amostras testadas no baseline para comparação
- Geração de descrições com temperatura 0.7
- Análise qualitativa da qualidade das respostas

**Critérios de avaliação**:
1. **Relevância**: Descrição relacionada ao título do produto
2. **Estrutura**: Formato e organização adequados
3. **Especificidade**: Detalhes e características do produto
4. **Coerência**: Fluidez e consistência do texto
5. **Domínio**: Adequação ao estilo Amazon de descrições de produtos

In [22]:
# Carregar modelo fine-tunado para demonstração
print("\n" + "="*80)
print("DEMONSTRAÇÃO DO MODELO FINE-TUNADO")
print("="*80)
print(f"⏳ Carregando modelo de {FINAL_MODEL_DIR}...\n")

finetuned_model = AutoModelForCausalLM.from_pretrained(
    FINAL_MODEL_DIR,
    torch_dtype=TORCH_DTYPE if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None
)

print("✓ Modelo fine-tunado carregado com sucesso!")
print(f"\nℹ Testando com {len(base_results)} amostras do conjunto de validação\n")
print("="*80 + "\n")

# Demonstrar geração de descrições
for i, (idx, row) in enumerate(base_results.iterrows(), 1):
    test_prompt = f"Descreva o produto com o seguinte título: {row['title']}\n\nDescrição:"
    
    print("-"*80)
    print(f"Exemplo {i}/{len(base_results)}")
    print("-"*80)
    print(f"Título do Produto: {row['title']}\n")
    
    print("Descrição Gerada pelo Modelo:")
    response = generate_response(finetuned_model, test_prompt)
    print(response)
    print("\n")

print("="*80)
print("✓ Demonstração concluída com sucesso!")
print("="*80 + "\n")


DEMONSTRAÇÃO DO MODELO FINE-TUNADO
⏳ Carregando modelo de ./modelo_finetuned...



Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.00it/s]


✓ Modelo fine-tunado carregado com sucesso!

ℹ Testando com 3 amostras do conjunto de validação


--------------------------------------------------------------------------------
Exemplo 1/3
--------------------------------------------------------------------------------
Título do Produto: The Divine Liturgy: A Commentary in the Light of the Fathers

Descrição Gerada pelo Modelo:
Descreva o produto com o seguinte título: The Divine Liturgy: A Commentary in the Light of the Fathers

Descrição: "This is a book that will be of great value to all who are interested in the Divine Liturgy, and especially to those who are preparing for Holy Communion."--The Orthodox Christian&#xA0;"This book is a valuable aid to the understanding of the Divine Liturgy, and will be of great use to all who are interested in the Divine Liturgy, and especially to those who are preparing for Holy Communion."--The Orthodox Christian"Dr. M. M.


-----------------------------------------------------------------------

## 6. Exemplos Adicionais

Demonstração com títulos de produtos de diferentes categorias.

In [23]:
# Exemplos adicionais de diferentes categorias
demo_titles = [
    "Wireless Bluetooth Headphones with Noise Cancellation",
    "Stainless Steel Water Bottle 500ml",
    "USB-C Fast Charging Cable 2m",
    "Organic Green Tea 100 Bags",
    "Gaming Mouse RGB LED Backlit"
]

print("\n" + "="*80)
print("EXEMPLOS ADICIONAIS DE GERAÇÃO")
print("="*80 + "\n")

for i, title in enumerate(demo_titles, 1):
    prompt = f"Descreva o produto com o seguinte título: {title}\n\nDescrição:"
    
    print("-"*80)
    print(f"Exemplo {i}/{len(demo_titles)}")
    print("-"*80)
    print(f"Título: {title}\n")
    
    response = generate_response(finetuned_model, prompt, max_new_tokens=150)
    description = response.split("Descrição:")[-1].strip()
    
    print("Descrição Gerada:")
    print(description)
    print("\n")

print("="*80)
print("✓ Geração de exemplos concluída!")
print("="*80)


EXEMPLOS ADICIONAIS DE GERAÇÃO

--------------------------------------------------------------------------------
Exemplo 1/5
--------------------------------------------------------------------------------
Título: Wireless Bluetooth Headphones with Noise Cancellation

Descrição Gerada:
Wireless Bluetooth Headphones with Noise Cancellation. These wireless Bluetooth headphones are designed to provide the ultimate listening experience. With the ability to pair with any Bluetooth device, these headphones are perfect for listening to music, podcasts, or audiobooks on the go. The noise cancellation feature allows you to block out background noise and focus on your audio. The high-quality sound delivers crystal clear audio, while the comfortable design ensures that you can wear them for extended periods of time without discomfort. The included charging cable and USB adapter make it easy to charge the headphones and use them with your computer or other USB devices. These wireless Bluetooth he

## 7. Conclusão

Este notebook demonstrou o processo completo de fine-tuning de um foundation model conforme os requisitos do Tech Challenge.

### Etapas Realizadas

**1. Preparação do Dataset**
- Dataset AmazonTitles-1.3MM carregado e processado
- 2.248.619 registros originais (2.2M+)
- 1.315.931 registros após limpeza (58.5% retidos, 932.688 removidos)
- Amostra de 20.000 registros selecionada (1.52% do dataset limpo)
- Split 90/10 treino/validação (18.000/2.000)
- Prompts formatados com título e descrição do produto

**2. Teste do Modelo Base (Baseline)**
- Modelo Phi-4-mini-instruct (3.8B parâmetros) testado pré-treinamento
- 3 amostras aleatórias avaliadas para estabelecer baseline
- Respostas documentadas para comparação posterior

**3. Execução do Fine-Tuning**
- Técnica LoRA (Parameter-Efficient Fine-Tuning) aplicada
- Apenas 0.0819% dos parâmetros treinados (3.145.728 de 3.839.167.488)
- Configuração otimizada: rank=16, alpha=32, dropout=0.1
- 2 épocas, 4.500 steps (2.250 por época)
- Duração aproximada de 40-60 minutos
- Precisão FP16 para velocidade máxima de treinamento
- Loss final: 2.7119
- 45 checkpoints salvos (a cada 100 steps)

**4. Geração de Respostas**
- Modelo treinado testado com as mesmas amostras do baseline
- Demonstração com 5 exemplos adicionais de diferentes categorias
- Modelo gera descrições coerentes e relevantes a partir dos títulos

### Parâmetros de Treinamento

| Parâmetro | Valor |
|-----------|-------|
| Modelo Base | microsoft/Phi-4-mini-instruct (3.8B) |
| Dataset Original | AmazonTitles-1.3MM (2.248.619 registros) |
| Dataset Limpo | 1.315.931 registros (58.5% retidos) |
| Amostra Treinamento | 20.000 registros (1.52% do limpo) |
| Split Treino/Validação | 18.000 / 2.000 (90% / 10%) |
| Técnica | LoRA (r=16, alpha=32, dropout=0.1) |
| Parâmetros Treináveis | 3.145.728 (0.0819% do total) |
| Parâmetros Totais | 3.839.167.488 |
| Épocas | 2 |
| Steps Totais | 4.500 |
| Batch Size por Device | 2 |
| Gradient Accumulation | 4 |
| Batch Size Efetivo | 8 |
| Learning Rate | 5e-5 |
| Max Length | 128 tokens |
| Precisão | FP16 |
| Hardware | NVIDIA RTX 2000 Ada (16GB) |
| Tempo de Treinamento | ~40-60 minutos |
| Loss Final | 2.7119 |

### Resultados

✅ **Fine-tuning executado com sucesso**
- Modelo especializado em gerar descrições de produtos
- Respostas contextualmente relevantes aos títulos fornecidos
- Formato e estilo adequados para descrições de e-commerce
- Pronto para demonstração em vídeo

### Arquivos Gerados

- `modelo_finetuned/`: Adaptadores LoRA e tokenizer
- `outputs/`: 45 checkpoints de treinamento (a cada 100 steps)
- `checkpoints/`: Melhores modelos salvos durante treinamento

### Conformidade com Tech Challenge

Este projeto atende todos os requisitos do Tech Challenge Fase 3:
1. ✅ Fine-tuning de foundation model (Phi-4-mini-instruct)
2. ✅ Uso do dataset AmazonTitles-1.3MM (arquivo trn.json)
3. ✅ Colunas title e content processadas
4. ✅ Teste do modelo antes do treinamento (baseline)
5. ✅ Documentação detalhada do processo
6. ✅ Geração de respostas baseadas em títulos de produtos
7. ✅ Código-fonte completo e reproduzível